In [ ]:
# --- Imports ---
import os
import random
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.metrics import classification_report, accuracy_score
import re

In [ ]:
# --- Configuration ---
MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct" 
CSV_PATH = r"C:\Users\stdFurqan\Desktop\Sindhi (politics)\split_30.csv"
TEXT_COL = "News Headlines"
LABEL_COL = "Aspect Category"
LABELS = ["Crime", "Sports", "Politics"]
SEED = 20
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# --- Reproducibility ---
def set_seed(seed=20):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
set_seed(SEED)

In [ ]:
# ✅ Local path where model is stored
MODEL_PATH = r"C:\Users\stdFurqan\Downloads\lama_models_download\LAMA_3.2(1b)"

print(f"🧠 Loading model from {MODEL_PATH} ...")

# ✅ Load tokenizer from local folder
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# ✅ Load model from local folder
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    torch_dtype=torch.bfloat16,   # ✅ best for 40-series
    device_map="auto"
)

model.config.pad_token_id = tokenizer.pad_token_id
model.eval()

print("✅ Model loaded successfully from local directory!")

In [ ]:


# --- Load dataset ---
df = pd.read_csv(CSV_PATH)
texts = df[TEXT_COL].astype(str).tolist()
true_labels = df[LABEL_COL].astype(str).tolist()

PROMPT_TEMPLATE = (
    "Classify the following news headline into one of these categories: Crime, Sports, or Politics.\n"
    "Reply with only one word: Crime, Sports, or Politics.\n\n"
    "Headline: {text}\n\nLabel:"
)

pattern = re.compile(r"\b(Crime|Sports|Politics)\b", re.IGNORECASE)

def extract_label(output):
    m = pattern.search(output)
    return m.group(1).capitalize() if m else "Unknown"

pred_labels = []
batch_size = 4  # smaller batch to fit in T4 VRAM
for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    prompts = [PROMPT_TEMPLATE.format(text=t) for t in batch]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=6,
            do_sample=False,
            temperature=0.0,
            top_p=1.0,
            pad_token_id=tokenizer.eos_token_id,
        )

    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    for prompt, full_out in zip(prompts, decoded):
        gen = full_out[len(prompt):].strip() if full_out.startswith(prompt) else full_out
        pred_labels.append(extract_label(gen))

# --- Evaluation ---
report = classification_report(true_labels, pred_labels, labels=LABELS, digits=4, zero_division=0)
acc = accuracy_score(true_labels, pred_labels)
print("\n📊 Classification Report:\n")
print(report)
print(f"Accuracy: {acc:.4f}")